In [1]:
!pip install cmake 'gym[atari]' scipy

     |████████████████████████████████| 21.5 MB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 35.5 MB/s eta 0:00:01
     |████████████████████████████████| 28.5 MB 23.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 14.0 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616825 sha256=d40aca3893eb7d59a719831e26a5b4fc3b69856f58048b261827817e4f4bcbf3
  Stored in directory: /root/.cache/pip/wheels/b3/50/6c/0a82c1358b4da2dbd9c1bb17e0f89467db32812ab236dbf6d5
Successfully built gym
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import gym

env = gym.make("Taxi-v3").env

In [3]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [4]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [5]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 48.5 s, sys: 5.83 s, total: 54.3 s
Wall time: 52.2 s


In [6]:
next_state, reward, done, info

(410, 20, True, {'prob': 1.0})

In [7]:
old_value, new_value

(19.99999999999999, 19.99999999999999)

In [8]:
q_table[308, 0]

-1.450240000000004

In [9]:
epochs, penalties, reward

(19, 0, 20)

In [10]:
state = env.encode(3, 0, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 308
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


In [11]:
# display the action values for South, North, East, West, Pickup, Dropoff
q_table[state]

array([ -1.45024   ,  -2.12208629,  -1.87014403,  -1.870144  ,
       -10.87013499, -10.87014342])

In [12]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.45
Average penalties per episode: 0.0


In [15]:
#taken from https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py
num_states = 500

num_actions = 6
P = {
    state: {action: [] for action in range(num_actions)}
    for state in range(num_states)
}

In [16]:
P

{0: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 1: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 2: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 3: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 4: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 5: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 6: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 7: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 8: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 9: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 10: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 11: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 12: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 13: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 14: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 15: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 16: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 17: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 18: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 19: {0: [], 1: [], 2: [], 3: [], 4: [], 5: []},
 20: {0: [], 1: [], 2: [], 3: 